- try stratifiedkfold

In [1]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn import preprocessing
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.metrics import average_precision_score
from sklearn.model_selection import StratifiedKFold
tqdm_notebook.pandas(desc="progress: ")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  
/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
DIR = "../input/20201115at/"
train = pd.read_csv(DIR+"train.csv")
test = pd.read_csv(DIR+"test.csv")
campaign = pd.read_csv(DIR+"campaign.csv")

#advertiser_video = pd.read_csv(DIR+"advertiser_video.csv")
#map_game_feed_native_video_assets = pd.read_csv(DIR+'map_game_feed_native_video_assets.csv')

#advertiser_converted_video = pd.read_csv(DIR+"advertiser_converted_video.csv")

# preprocess

In [3]:
# with campaign
train = train.merge(campaign, left_on="campaign_id", right_on = "id", how="left").drop(["mst_advertiser_id", "id"], axis=1)
test = test.merge(campaign, left_on="campaign_id", right_on = "id", how="left").drop(["mst_advertiser_id", "id"], axis=1)

# with map_gae_feed_native_video_assets
#train = train.merge(map_game_feed_native_video_assets, left_on=["game_feed_id"], right_on = ["mst_game_feed_id"], how="left").drop(["mst_game_feed_id"], axis=1)
#test = test.merge(map_game_feed_native_video_assets, left_on=["game_feed_id"], right_on = ["mst_game_feed_id"], how="left").drop(["mst_game_feed_id"], axis=1)

# with advertiser_video horizontal
#horizontal = advertiser_video.copy()
#horizontal.columns = [f"horizontal_{c}" if c not in ["id", "mst_advertiser_id"] else c for c in horizontal.columns]
#train = pd.merge(train, horizontal[["id", "mst_advertiser_id", "horizontal_duration"]], left_on=["horizontal_mst_advertiser_video_id", "advertiser_id"], 
#               right_on=["id", "mst_advertiser_id"], how="left").drop(columns=["id", "mst_advertiser_id"])
#test = pd.merge(test, horizontal[["id", "mst_advertiser_id", "horizontal_duration"]], left_on=["horizontal_mst_advertiser_video_id", "advertiser_id"], 
#               right_on=["id", "mst_advertiser_id"], how="left").drop(columns=["id", "mst_advertiser_id"])

In [4]:
train["imp_at"] = pd.to_datetime(train["imp_at"])
test["imp_at"] = pd.to_datetime(test["imp_at"])

In [5]:
train.drop(['click_or_view_at', 'cv_at','last_cv_interval', 'last_paid_interval'], axis=1, inplace=True)

In [6]:
object_feats = [i for i in train.columns if "object" == str(train[i].dtype)]
train[object_feats] = train[object_feats].fillna("-999")
test[object_feats] = test[object_feats].fillna("-999")

for i in object_feats:
    print(i)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(pd.concat([train[i], test[i]])))
    train[i] = lbl.transform(list(train[i]))
    test[i] = lbl.transform(list(test[i]))

request_id
advertiser_id
app_id
campaign_id
country_code
game_feed_id
game_template_id
media_app_id
os_version
uid
video_template_id
mst_advertiser_order_id


# feature engineering

In [7]:
train["type"] = "train"
test["type"] = "test"
all_df = pd.concat([train, test])
all_df = all_df.sort_values(["uid", "imp_at"]).reset_index(drop=True)

In [8]:
all_df["count"] = 1
all_df["count"] = all_df.groupby("uid")["count"].cumsum()

In [9]:
#all_df["prev_app"] = all_df.groupby("uid")["app_id"].progress_apply(lambda x: x.shift(1))

In [10]:
train = all_df[all_df.type=="train"].drop("type", axis=1).reset_index(drop=True)
test = all_df[all_df.type=="test"].drop("type", axis=1).reset_index(drop=True)

In [11]:
train = train.sort_values("imp_at").reset_index(drop=True)
test = test.sort_values("imp_at").reset_index(drop=True)

# separation for validation

In [12]:
train.drop(['request_id'], axis=1, inplace=True)
test.drop(['request_id'], axis=1, inplace=True)

In [13]:
#mask = train.imp_at > "2020-05-21"
#valid = train.loc[mask].copy()
#train = train.loc[~mask]

In [14]:
remove_features = ["imp_at","game_feed_asset_type_id","country_code","pos","video_template_id"]
    
train = train.drop(remove_features, axis=1)
#valid = valid.drop(remove_features, axis=1)
test = test.drop(remove_features, axis=1)

In [15]:
test = test.drop("target", axis=1)

In [16]:
train.shape, test.shape #valid.shape,

((1997595, 28), (390095, 27))

In [17]:
train.head()

,target,adnw_id,adspot_id,adspot_video_format_id,advertiser_id,app_id,auction_type_id,campaign_id,category_id,first_login_interval,...,login_frequency,max_login_interval,media_app_id,os,os_version,uid,user_type_id,mst_advertiser_order_id,mst_user_type_id,count
0,0.0,4,1111,2.0,20,14872,1.0,403,8497,1490644.0,...,10.0,2607.0,0,1,87,273693,1,37,35,1
1,0.0,28,1081,2.0,121,8301,2.0,91,0,790412.0,...,2.0,13540.0,8995,2,152,427139,2,166,37,1
2,0.0,42,1123,3.0,5,12948,1.0,114,0,1506105.0,...,9.0,33725.0,347,1,55,1005266,1,27,35,1
3,0.0,8,1113,2.0,22,12379,1.0,75,0,184283.0,...,4.0,7377.0,6145,1,91,492541,1,159,38,1
4,0.0,22,1061,NaN,135,196,4.0,331,0,1999495.0,...,5.0,164759.0,2190,1,71,698493,1,56,35,1


# modelling

In [18]:
object_feats = [i for i in object_feats if i not in remove_features and i != "request_id"]

In [19]:
n_folds = 5
def pr_auc_metric(y_predicted, y_true):
    return 'pr_auc', average_precision_score(y_true.get_label(), y_predicted), True

def modelling(seed):
    print("seed:", seed)
    lgbm_params = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 
               'tree_learner': 'serial', 'learning_rate': 0.1, "num_leaves": 10, 'random_seed':seed,'max_depth': 5}
    
    y = train.target.copy()
    X = train.drop(['target'],axis=1).copy()
    
    skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
    
    valid = np.zeros([X.shape[0]])
    features_list = [i for i in X.columns]
    pred_value = np.zeros(test.shape[0])
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    
    for i , (train_index, val_index) in enumerate(skf.split(X, y)):
        print("fold: ", i)
        
        X_train = X.iloc[train_index,:]
        y_train = y.iloc[train_index]

        X_valid = X.iloc[val_index,:]
        y_valid = y.iloc[val_index]
        
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
        
        model = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=20,verbose_eval = 10,
                          feval=pr_auc_metric, categorical_feature = sorted(object_feats))
        
        valid_predict = model.predict(X_valid, num_iteration = model.best_iteration)
        valid[val_index] = valid_predict
        pred_value += model.predict(test, num_iteration = model.best_iteration)
        feature_importance_df["Fold_"+str(i+1)] = model.feature_importance()
    
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
    
    score = average_precision_score(y_valid, valid_predict)
    print("average precision score = {}".format(score))
    
    return pred_value, valid, feature_importance_df

validation = np.zeros([train.shape[0]])
pred_value = np.zeros([test.shape[0]])
seeds = [0]
for seed_ in seeds:
    test_pred, oof_predict, feature_importance_df = modelling(seed_)
    validation += oof_predict / len(seeds)
    pred_value += test_pred / len(seeds)

score = average_precision_score(train.target, validation)
print(score)

seed: 0
fold:  0


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['advertiser_id', 'app_id', 'campaign_id', 'game_feed_id', 'game_template_id', 'media_app_id', 'mst_advertiser_order_id', 'os_version', 'uid']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 20 rounds
[10]	training's pr_auc: 0.215284	valid_1's pr_auc: 0.210971
[20]	training's pr_auc: 0.227006	valid_1's pr_auc: 0.220222
[30]	training's pr_auc: 0.236822	valid_1's pr_auc: 0.227529
[40]	training's pr_auc: 0.24521	valid_1's pr_auc: 0.231816
[50]	training's pr_auc: 0.251765	valid_1's pr_auc: 0.234534
[60]	training's pr_auc: 0.257294	valid_1's pr_auc: 0.236427
[70]	training's pr_auc: 0.261999	valid_1's pr_auc: 0.237878
[80]	training's pr_auc: 0.265899	valid_1's pr_auc: 0.23901
[90]	training's pr_auc: 0.269118	valid_1's pr_auc: 0.239755
[100]	training's pr_auc: 0.27215	valid_1's pr_auc: 0.240373
[110]	training's pr_auc: 0.274805	valid_1's pr_auc: 0.240811
[120]	training's pr_auc: 0.277583	valid_1's pr_auc: 0.241405
[130]	training's pr_auc: 0.279367	valid_1's pr_auc: 0.241598
[140]	training's pr_auc: 0.281236	valid_1's pr_auc: 0.241789
[150]	training's pr_auc: 0.283563	valid_1's pr_auc: 0.242153
[160]	training's pr_auc: 0.285031	va

In [20]:
feature_importance_df.sort_values("Average", ascending=False)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
11,game_feed_id,422,414,371,434,314,391.0,43.972719,0.112462
4,app_id,385,382,346,425,314,370.4,37.696684,0.101773
6,campaign_id,243,251,248,272,233,249.4,12.846789,0.051511
22,uid,186,219,175,207,156,188.6,22.437469,0.118969
19,media_app_id,176,166,159,177,126,160.8,18.626862,0.115839
3,advertiser_id,62,74,60,75,74,69.0,6.572671,0.095256
26,count,68,76,63,75,56,67.6,7.499333,0.110937
8,first_login_interval,48,62,51,65,49,55.0,7.071068,0.128565
16,last_login_interval,61,58,44,53,42,51.6,7.499333,0.145336
24,mst_advertiser_order_id,59,46,48,45,34,46.4,7.964923,0.171658


# submission

In [21]:
lgb_sub = pd.read_csv(DIR + "atmaCup7__sample_submission.csv")
lgb_sub["target"] = pred_value
lgb_sub.to_csv("atmacup7_lgb"+str(score)[:-10]+".csv", index = False)